# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 54 new papers today
          19 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/18 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.08098


extracting tarball to tmp_2212.08098... done.
Retrieving document from  https://arxiv.org/e-print/2212.08129


/tmp/ipykernel_2082/4030337529.py:34: LatexWarning: 2212.08129 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2212.08185


...

Retrieving document from  https://arxiv.org/e-print/2212.08274
Retrieving document from  https://arxiv.org/e-print/2212.08325
Retrieving document from  https://arxiv.org/e-print/2212.08419
Retrieving document from  https://arxiv.org/e-print/2212.08436


/tmp/ipykernel_2082/4030337529.py:34: LatexWarning: 2212.08436 did not run properly
<urlopen error [Errno 104] Connection reset by peer>
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/tmp/ipykernel_2082/4030337529.py:34: LatexWarning: 2212.08444 did not run properly
<urlopen error [Errno 104] Connection reset by peer>
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/tmp/ipykernel_2082/4030337529.py:34: LatexWarning: 2212.08453 did not run properly
<urlopen error [Errno 104] Connection reset by peer>
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/tmp/ipykernel_2082/4030337529.py:34: LatexWarning: 2212.08477 did not run properly
<urlopen error [Errno 104] Connection reset by peer>
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/tmp/ipykernel_2082/4030337529.py:34: LatexWarning: 2212.08504 did not run properly
<urlopen error [Errno 104] Connection reset by peer>
  warnings.

Retrieving document from  https://arxiv.org/e-print/2212.08444
Retrieving document from  https://arxiv.org/e-print/2212.08453
Retrieving document from  https://arxiv.org/e-print/2212.08477
Retrieving document from  https://arxiv.org/e-print/2212.08504
Retrieving document from  https://arxiv.org/e-print/2212.08517


Retrieving document from  https://arxiv.org/e-print/2212.08528


/tmp/ipykernel_2082/4030337529.py:34: LatexWarning: 2212.08517 did not run properly
<urlopen error [Errno 104] Connection reset by peer>
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/tmp/ipykernel_2082/4030337529.py:34: LatexWarning: 2212.08528 did not run properly
<urlopen error [Errno 104] Connection reset by peer>
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.08098-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.08098) | **A Dark Matter Trigger for Early Dark Energy Coincidence**  |
|| <mark>Meng-Xiang Lin</mark>, Evan McDonough, J. Colin Hill, Wayne Hu |
|*Appeared on*| *2022-12-19*|
|*Comments*| *12 pages, 6 figures*|
|**Abstract**| Early dark energy (EDE), whose cosmological role is localized in time around the epoch of matter-radiation equality in order to resolve the Hubble tension, introduces a new coincidence problem: why should the EDE dynamics occur near equality if EDE is decoupled from both matter and radiation? The resolution of this problem may lie in an {\it early dark sector} (EDS), wherein the dark matter mass is dependent on the EDE scalar field. Concretely, we consider a Planck-suppressed coupling of EDE to dark matter, as would naturally arise from breaking of the global $U(1)$ shift symmetry of the former by quantum gravity effects. With a sufficiently flat potential, the rise to dominance of dark matter at matter-radiation equality itself triggers the rolling and subsequent decay of the EDE. We show that this {\it trigger} EDS (tEDS) model can naturally resolve the EDE coincidence problem at the background level without any fine tuning of the coupling to dark matter or of the initial conditions. When fitting to current cosmological data, including that from the local distance ladder and the low-redshift amplitude of fluctuations, the tEDS maximum-likelihood model performs comparably to EDE for resolving the Hubble tension, achieving $H_0 =71.2$ km/s/Mpc. However, fitting the \emph{Planck} cosmic microwave background data requires a specific range of initial field positions to balance the scalar field fluctuations that drive acoustic oscillations, providing testable differences with other EDE models. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.08185-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.08185) | **Tracking hidden magnetospheric fluctuations in accretion-powered pulsars  with a Kalman filter**  |
|| A. Melatos, N. J. O'Neill, <mark>P. M. Meyers</mark>, J. O'Leary |
|*Appeared on*| *2022-12-19*|
|*Comments*| *49 pages, 6 figures, accepted for publication in The Astrophysical Journal*|
|**Abstract**| X-ray flux and pulse period fluctuations in an accretion-powered pulsar convey important information about the disk-magnetosphere interaction. It is shown that simultaneous flux and period measurements can be analysed with a Kalman filter based on the standard magnetocentrifugal accretion torque to generate accurate time-dependent estimates of three hidden state variables, which fluctuate stochastically and cannot be measured directly: the mass accretion rate, the Maxwell stress at the disk-magnetosphere boundary, and the radiative efficiency of accretion onto the stellar surface. The inferred fluctuation statistics carry implications for the physics of hydromagnetic instabilities at the disk-magnetosphere boundary and searches for continuous gravitational radiation from low-mass X-ray binaries. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.08194-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.08194) | **On the gamma-ray emission from the core of the Sagittarius dwarf galaxy**  |
|| Addy J. Evans, et al. -- incl., <mark>Tim Linden</mark> |
|*Appeared on*| *2022-12-19*|
|*Comments*| *12 pages, 9 figures. To be submitted to MNRAS -- comments welcome*|
|**Abstract**| We use data from the Large Area Telescope onboard the Fermi gamma-ray space telescope (Fermi-LAT) to analyze the faint gamma-ray source located at the center of the Sagittarius (Sgr) dwarf spheroidal galaxy. In the 4FGL-DR3 catalog, this source is associated with the globular cluster, M54, which is coincident with the dynamical center of this dwarf galaxy. We investigate the spectral energy distribution and spatial extension of this source, with the goal of testing two hypotheses: (1) the emission is due to millisecond pulsars within M54, or (2) the emission is due to annihilating dark matter from the Sgr halo. For the pulsar interpretation, we consider a two-component model which describes both the lower-energy magnetospheric emission and possible high-energy emission arising from inverse Compton scattering. We find that this source has a point-like morphology at low energies, consistent with magnetospheric emission, and find no evidence for a higher-energy component. For the dark matter interpretation, we find that this signal favors a dark matter mass of $m_{\chi} = 29.6 \pm 5.8$ GeV and an annihilation cross section of $\sigma v = (2.1 \pm 0.59) \times 10^{-26} \,\text{cm}^3/$s for the $b \bar{b}$ channel (or $m_{\chi} = 8.3 \pm 3.8$ GeV and $\sigma v = (0.90 \pm 0.25) \times 10^{-26} \, \text{cm}^3/$s for the $\tau^+ \tau^-$ channel), when adopting a J-factor of $J=10^{19.6} \, \text{GeV}^2 \, \text{cm}^{-5}$. This parameter space is consistent with gamma-ray constraints from other dwarf galaxies and with dark matter interpretations of the Galactic Center Gamma-Ray Excess. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.08504-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.08504) | **Morphological Classification of Radio Galaxies with wGAN-supported  Augmentation**  |
|| Lennart Rustige, et al. -- incl., <mark>Frank Gaede</mark> |
|*Appeared on*| *2022-12-19*|
|*Comments*| *12 pages, 7+2 figures, 1+2 tables. Submitted, comments welcome*|
|**Abstract**| Machine learning techniques that perform morphological classification of astronomical sources often suffer from a scarcity of labelled training data. Here, we focus on the case of supervised deep learning models for the morphological classification of radio galaxies, which is particularly topical for the forthcoming large radio surveys. We demonstrate the use of generative models, specifically Wasserstein GANs (wGANs), to generate data for different classes of radio galaxies. Further, we study the impact of augmenting the training data with images from our wGAN on three different classification architectures. We find that this technique makes it possible to improve models for the morphological classification of radio galaxies. A simple Fully Connected Neural Network (FCN) benefits most from including generated images into the training set, with a considerable improvement of its classification accuracy. In addition, we find it is more difficult to improve complex classifiers. The classification performance of a Convolutional Neural Network (CNN) can be improved slightly. However, this is not the case for a Vision Transformer (ViT). |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error <urlopen error [Errno 104] Connection reset by peer></p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.08517-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.08517) | **The hot gas distribution, X-ray luminosity and baryon budget in the  L-Galaxies semi-analytic model of galaxy formation**  |
|| Wenxin Zhong, et al. -- incl., <mark>Prateek Sharma</mark> |
|*Appeared on*| *2022-12-19*|
|*Comments*| *18 pages, 15 figures, accepted for publication in MNRAS*|
|**Abstract**| Hot ionized gas is important in the baryon cycle of galaxies and contributes the majority of their ``missing baryons''. Until now, most semi-analytic models of galaxy formation have paid little attention to hot gaseous haloes and their X-ray emission. In this paper, we adopt the one-dimensional model from Sharma et al. instead of the isothermal sphere to describe the radial distribution of hot gas in the L-Galaxies semi-analytic model. The hot gas halo can be divided into two parts according to the ratio of the local thermal instability time-scale and the free-fall time-scale: a cool core with $t_{\rm TI}/t_{\rm ff}=10$ and a stable outer halo with $t_{\rm TI}/t_{\rm ff}>10$. We update the prescriptions of cooling, feedback and stripping based on the new hot gas profiles, and then reproduce several X-ray observational results, like the radial profiles of hot gas density, and the scaling relations of X-ray luminosity and temperature. We find: (1) Consistent with observations, flatter density profiles in halo centers produce lower X-ray emission than an isothermal sphere; (2) Cool core regions prone to precipitation have higher gas temperature than the virial temperature, and a larger $T_{\rm X}/T_{\rm 200}$ ratio in smaller haloes leads to a steeper slope in the $L_{\rm X}-T_{\rm X}$ relation; (3) The ionized gas in the unbounded reservoir and low temperature intergalactic gas in low mass haloes could be the main components of the halo ``missing baryons''. Our model outputs can predict the observations of hot gas in the nearby universe and produce mock surveys of baryons probed by future X-ray telescopes. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error <urlopen error [Errno 104] Connection reset by peer></p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.08528-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.08528) | **The TRAPUM L-band survey for pulsars in Fermi-LAT gamma-ray sources**  |
|| C. J. Clark, et al. -- incl., <mark>M. Kramer</mark> |
|*Appeared on*| *2022-12-19*|
|*Comments*| *17 pages, 8 figures, accepted for publication in MNRAS*|
|**Abstract**| More than 100 millisecond pulsars (MSPs) have been discovered in radio observations of gamma-ray sources detected by the Fermi Large Area Telescope (LAT), but hundreds of pulsar-like sources remain unidentified. Here we present the first results from the targeted survey of Fermi-LAT sources being performed by the Transients and Pulsars with MeerKAT (TRAPUM) Large Survey Project. We observed 79 sources identified as possible gamma-ray pulsar candidates by a Random Forest classification of unassociated sources from the 4FGL catalogue. Each source was observed for 10 minutes on two separate epochs using MeerKAT's L-band receiver (856-1712 MHz), with typical pulsed flux density sensitivities of $\sim$100$\,\mu$Jy. Nine new MSPs were discovered, eight of which are in binary systems, including two eclipsing redbacks and one system, PSR J1526$-$2744, that appears to have a white dwarf companion in an unusually compact 5 hr orbit. We obtained phase-connected timing solutions for two of these MSPs, enabling the detection of gamma-ray pulsations in the Fermi-LAT data. A follow-up search for continuous gravitational waves from PSR J1526$-$2744 in Advanced LIGO data using the resulting Fermi-LAT timing ephemeris yielded no detection, but sets an upper limit on the neutron star ellipticity of $2.45\times10^{-8}$. We also detected X-ray emission from the redback PSR J1803$-$6707 in data from the first eROSITA all-sky survey, likely due to emission from an intra-binary shock. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error <urlopen error [Errno 104] Connection reset by peer></p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.08249-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.08249) | **Phase-space Properties and Chemistry of the Sagittarius Stellar Stream  Down to the Extremely Metal-poor ($\rm[Fe/H] \lesssim -3$) Regime**  |
|| <mark>Guilherme Limberg</mark>, et al. -- incl., <mark>Young Sun Lee</mark> |
|*Appeared on*| *2022-12-19*|
|*Comments*| *Submitted to ApJ. Comments are welcomed and sincerely appreciated!*|
|**Abstract**| In this work, we study the phase-space and chemical properties of Sagittarius (Sgr) stream, the tidal tails produced by the ongoing destruction of Sgr dwarf spheroidal (dSph) galaxy, focusing on its very metal-poor (VMP; $\rm[Fe/H] < -2$) content. We combine spectroscopic and astrometric information from SEGUE and $Gaia$ EDR3, respectively, with data products from a new large-scale run of $\texttt{StarHorse}$ spectro-photometric code. Our selection criteria yields ${\sim}1600$ stream members, including ${>}200$ VMP stars. We find the leading arm ($b>0$) of Sgr stream to be more metal-poor, by ${\sim}0.2$ dex, than the trailing one ($b<0$). With a subsample of turnoff and subgiant stars, we estimate this substructure's stellar population to be ${\sim}1$ Gyr older than the thick disk's. With the aid of an $N$-body model of the Sgr system, we verify that simulated particles stripped earlier (${>}2$ Gyr ago) have present-day phase-space properties similar to lower-metallicity stream stars. Conversely, those stripped more recently (${<}2$ Gyr) are preferentially more akin to metal-rich ($\rm[Fe/H] > -1$) members of the stream. Such correlation between kinematics and chemistry can be explained by the existence of a dynamically hotter, less centrally-concentrated, and more metal-poor population in Sgr dSph prior to its disruption, implying that this galaxy was able to develop a metallicity gradient before its accretion. Finally, we discovered several carbon-enhanced metal-poor ($\rm[C/Fe] > +0.7$ and $\rm[Fe/H] \leq -1.5$) stars in Sgr stream, which is in tension with current observations of its remaining core where such objects are not found. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 104] Connection reset by peer</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.08129-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.08129) | **Micro-arcsecond Astrometry Technology: Detector and Field Distortion  Calibration**  |
|| Michael Shao, et al. -- incl., <mark>Chengxing Zhai</mark> |
|*Appeared on*| *2022-12-19*|
|*Comments*| *16 pages, 10 figures, to be submitted to PASP*|
|**Abstract**| Microarcsecond (uas) astrometry provides an indispensable way to survey earth-like exoplanets and fully characterize the orbits and masses for assessing their habitability. Highly accurate astrometric measurements can also probe the nature of dark matter, the primordial universe, black holes, and neutron stars for new astrophysics. This paper presents technology for calibrating array detectors and field distortions to achieve narrow field uas astrometry using a 6 m telescope with a focal plane array detector. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

133  publications in the last 7 days.
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/html/2211.15288.md
	 _build/html/2211.15224.md
	 _build/html/2211.14543.md
	 _build/html/2211.14330.md
	 _build/html/2211.14322.md
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers